In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive

# Google Drive'ı bağla
drive.mount('/content/drive')

# Veri yolunu Google Drive'daki doğru yol ile ayarla
data_dir = '/content/drive/My Drive/Colab Notebooks/money detection project/dataset_gradcam'  # Tüm verilerin bulunduğu ana klasör

batch_size = 32
img_width, img_height = 128, 128  # Görüntü boyutları

# Data augmentation ve validation split için ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # %20 doğrulama, %80 eğitim için ayrılacak
)

# Eğitim verisi için generator
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Eğitim için subset
)

# Doğrulama verisi için generator
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Doğrulama için subset
)

# Sınıfların sayısını kontrol et
print("Sınıf Sayısı:", train_generator.num_classes)
print("Sınıf Etiketleri:", train_generator.class_indices)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 7443 images belonging to 5 classes.
Found 1858 images belonging to 5 classes.
Sınıf Sayısı: 5
Sınıf Etiketleri: {'1.emisyon': 0, '2.emisyon': 1, '3.emisyon': 2, '4.emisyon': 3, '5.emisyon': 4}


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, TimeDistributed, Flatten, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

img_width, img_height = 128, 128  # Görüntü boyutları
lstm_units = 64  # LSTM hücre sayısı
dropout_rate = 0.5  # Dropout oranı
learning_rate = 0.001  # Öğrenme oranı

# CNN + LSTM Modeli
model = Sequential()

# Removing TimeDistributed and adjusting input_shape
# input_shape is now (img_width, img_height, 3) for individual images
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

# If you still need LSTM for some temporal features,
# you might need to reshape the output of Flatten to
# have a time dimension before feeding it to LSTM.
# This would require further modifications to your data and model architecture.

# Çıkış katmanı
model.add(Dense(128, activation='relu'))
model.add(Dense(train_generator.num_classes, activation='softmax'))  # Sınıf sayısını burada alıyoruz

# Modeli derleme
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli özetle
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 57600)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       7,372,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,392,965 (28.20 MB)

 Trainable params: 7,392,965 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Early stopping callback eklenebilir
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Define the number of epochs
epochs = 10  # You can change this to your desired number of epochs

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


232/232 ━━━━━━━━━━━━━━━━━━━━ 4174s 18s/step - accuracy: 0.4688 - loss: 1.4164 - val_accuracy: 0.5280 - val_loss: 1.5486
Epoch 2/10
  1/232 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.7500 - loss: 0.7812

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


232/232 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7500 - loss: 0.7812 - val_accuracy: 1.0000 - val_loss: 0.8456
Epoch 3/10
232/232 ━━━━━━━━━━━━━━━━━━━━ 198s 633ms/step - accuracy: 0.6539 - loss: 0.8654 - val_accuracy: 0.5485 - val_loss: 1.4554
Epoch 4/10
232/232 ━━━━━━━━━━━━━━━━━━━━ 10s 42ms/step - accuracy: 0.6562 - loss: 0.8852 - val_accuracy: 1.0000 - val_loss: 0.2096
Epoch 5/10
232/232 ━━━━━━━━━━━━━━━━━━━━ 145s 612ms/step - accuracy: 0.7614 - loss: 0.6288 - val_accuracy: 0.6218 - val_loss: 1.5628
Epoch 6/10
232/232 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.8438 - loss: 0.4307 - val_accuracy: 1.0000 - val_loss: 0.4460
Epoch 7/10
232/232 ━━━━━━━━━━━━━━━━━━━━ 145s 614ms/step - accuracy: 0.8136 - loss: 0.4920 - val_accuracy: 0.6541 - val_loss: 1.5889
Epoch 8/10
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 103us/step - accuracy: 0.7812 - loss: 0.4615 - val_accuracy: 1.0000 - val_loss: 0.2426
Epoch 9/10
232/232 ━━━━━━━━━━━━━━━━━━━━ 201s 608ms/step - accuracy: 0.8447 - loss: 0.4083 - val_ac

In [7]:
from sklearn.metrics import classification_report, confusion_matrix

# Doğrulama setinde tahminler yapma
validation_generator.reset()
predictions = model.predict(validation_generator, steps=validation_generator.samples // batch_size + 1)
y_pred = predictions.argmax(axis=1)

# Gerçek etiketleri alma
y_true = validation_generator.classes

# Performans metriklerini hesaplama
print('Classification Report:')
print(classification_report(y_true, y_pred, target_names=validation_generator.class_indices.keys()))

# Confusion matrix (Karışıklık matrisi) ile inceleme
print('Confusion Matrix:')
print(confusion_matrix(y_true, y_pred))


59/59 ━━━━━━━━━━━━━━━━━━━━ 34s 563ms/step
Classification Report:
              precision    recall  f1-score   support

   1.emisyon       0.14      0.19      0.16       275
   2.emisyon       0.32      0.40      0.36       603
   3.emisyon       0.24      0.24      0.24       409
   4.emisyon       0.14      0.09      0.11       150
   5.emisyon       0.23      0.13      0.17       421

    accuracy                           0.25      1858
   macro avg       0.21      0.21      0.21      1858
weighted avg       0.24      0.25      0.24      1858

Confusion Matrix:
[[ 51 117  59  13  35]
 [112 240 133  29  89]
 [ 80 161  99  24  45]
 [ 21  69  31  14  15]
 [ 91 162  91  22  55]]


In [9]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score # Import necessary functions


# Test veya doğrulama verileri üzerinde tahmin yapma
# 'validation_generator' kullanarak tahminleri alıyoruz.
y_true = validation_generator.classes  # Gerçek etiketler
y_pred_prob = model.predict(validation_generator)  # Tahmin edilen olasılıklar

# En yüksek olasılığa göre sınıfları belirle
y_pred = np.argmax(y_pred_prob, axis=1)

# Metrikleri hesaplayın
precision = precision_score(y_true, y_pred, average='weighted')  # 'macro' veya 'micro' da kullanabilirsiniz
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
accuracy = accuracy_score(y_true, y_pred)

# Sonuçları yazdırın
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

59/59 ━━━━━━━━━━━━━━━━━━━━ 32s 547ms/step
Precision: 0.2395
Recall: 0.2492
F1 Score: 0.2397
Accuracy: 0.2492
